In [4]:
# libraries you will need
import datetime as dt
import numpy as np
import pandas as pd
import tables as tb
import requests
import fxcmpy
from fxcmpy import fxcmpy_tick_data_reader as tdr
import os
import gzip
from io import StringIO
from multiprocessing import  Pool
import gc

In [5]:
directory = "/home/ks/Templates/data/"

In [7]:
df1 = pd.DataFrame(columns = ['DateTime', 'Bid', 'Ask','Spread','DeltaT'])
df1.to_csv("/home/ks/Templates/ImportantData.csv",mode='w')

In [6]:
def parallelize_dataframe(df, func, n_cores=8):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [9]:
def Max(A):
    return A.Ask.max()
MAX=np.vectorize(Max)

In [10]:
def Min(A):
    return A.Bid.min()
MIN=np.vectorize(Min)

In [44]:
def modify(data_pd):
    data_pd.DateTime=pd.to_datetime(data_pd.DateTime).astype('int64')/1000000
    data_pd.sort_values(by=['DateTime'],ignore_index=True,inplace=True)
    data_pd['Spread'] = data_pd['Ask'].sub(data_pd['Bid'], axis = 0) 
    data_pd['DeltaT'] = data_pd['DateTime'].diff()
    data_pd['DeltaBid'] = data_pd['Bid'].diff()
    data_pd['DeltaAsk'] = data_pd['Ask'].diff()
    data_pd['DeltaSpread'] = data_pd['Spread'].diff()
    A=np.array_split(data_pd, 64)
    
    return data_pd
    
            


In [45]:
# ExtractTransformLoad script
for file in os.listdir(directory):
    if file.endswith('.gz')& file.startswith( 'EURGBP' ):        
        # extract gzip file and assign to Dataframe
        codec = 'utf-16'
        f = gzip.GzipFile(f'{directory}{file}')
        data = f.read()
        data_str = data.decode(codec)
        data_pd = pd.read_csv(StringIO(data_str))
        # pad missing zeros in microsecond field
        data_pd['DateTime'] = data_pd.DateTime.str.pad(26, side='right', fillchar='0')
        # assign Datetime column as index
        # sample start and end to determine date format
        data_pd=parallelize_dataframe(data_pd, modify)
        # Load data into single CSV
        data_pd.to_csv("/home/ks/Templates/ImportantData.csv",mode='a',header=False)
        #after appending in csv, clearing memory
        gc.collect()
        



In [ ]:
for file in os.listdir(directory):
    if file.endswith('.gz'):
        if os.path.getsize(directory+file) < 10000:
            print("the file",file," is corrupted")


# ExtractTransformLoad script
directory = "/home/ks/Templates/data/"
HDF5 = "/home/ks/Templates/data/H5.h5"

for file in os.listdir(directory):
    if file.endswith('.gz'):
        print(os.path.getsize(directory+file))
        if os.path.getsize(directory+file) > 10000:
            print(f"\nExtracting: {file}")
        
            # extract gzip file and assign to Dataframe
            codec = 'utf-16'
            f = gzip.GzipFile(f'{directory}{file}')
            #print(f)
            data = f.read()
            data_str = data.decode(codec)
            data_pd = pd.read_csv(StringIO(data_str))

            # pad missing zeros in microsecond field
            data_pd['DateTime'] = data_pd.DateTime.str.pad(26, side='right', fillchar='0')

            # assign Datetime column as index
            data_pd.set_index('DateTime', inplace=True)

            # sample start and end to determine date format
            sample1 = data_pd.index[1]
            sample2 = data_pd.index[-1]

            # determine datetime format and supply srftime directive
            for row in data_pd:
                if data_pd.index[3] == '/':
                    if sample1[0:2] == sample2[0:2]:
                        data_pd.index = pd.to_datetime(data_pd.index, format="%m/%d/%Y %H:%M:%S.%f")
                    elif sample1[3:5] == sample2[3:4]:
                        data_pd.index = pd.to_datetime(data_pd.index, format="%d/%m/%Y %H:%M:%S.%f")
                elif data_pd.index[5] == '/':
                    if sample1[9:11] == sample2[9:11]:
                        data_pd.index = pd.to_datetime(data_pd.index, format="%Y/%d/%m %H:%M:%S.%f")
                    elif sample[6:8] == sample2[6:8]:
                        data_pd.index = pd.to_datetime(data_pd.index, format="%Y/%m/%d %H:%M:%S.%f")

            #print("\nDATA SUMMARY:")
            #print(data_pd.info())

            # Load data into single CSV
            store = pd.HDFStore(HDF5)
            symbol = file[:6]
            store.append(symbol, data_pd, format='t') 
            store.flush()
            #print("\nH5 DATASTORE SUMMARY:")
            #print(store.info()+"\n"+"-"*75)
            store.close()
        else:
            print("the file",file," is corrupted")
            


